In [0]:
#Importamos las librerías necesarias
from pyspark.sql import SparkSession
#from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace, upper, date_format, current_date, add_months, when, col
from pyspark.sql.types import IntegerType, DoubleType
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [0]:
#variables
spark = SparkSession.builder.getOrCreate()

#Archivo cloud storage
name_bucket = "dmc_dde_barb"
#Path del lakehouse, capa bronze (origen)
#capa silver (destino)
path_lakehouse = f"gs://{name_bucket}/produccion/dmc"
path_bronze = f"{path_lakehouse}/bronze/empresas/"
path_silver = f"{path_lakehouse}/silver/empresas/"


In [0]:
df = spark.read.format("delta").option("header","true").load(path_bronze)
display(df)


ID,NOMBRE
1,Walmart
2,Microsoft
3,Apple
4,Toyota
5,Amazon
6,Google
7,Samsung
8,HP
9,IBM
10,Sony


In [0]:
df_t = df.withColumn('NOMBRE',upper(col('NOMBRE'))).withColumn("PERIODO",date_format(add_months(current_date(),-1),"yyyyMM"))

#display(df_t) 

df_c = df_t.withColumn('ID',col('ID').cast(IntegerType()))

display(df_c)


ID,NOMBRE,PERIODO
1,WALMART,202502
2,MICROSOFT,202502
3,APPLE,202502
4,TOYOTA,202502
5,AMAZON,202502
6,GOOGLE,202502
7,SAMSUNG,202502
8,HP,202502
9,IBM,202502
10,SONY,202502


In [0]:
df_c.write.mode('overwrite').partitionBy('PERIODO').format('delta').save(path_silver)